In [1]:
using Random
using Images
using FileIO

In [2]:
# Definição das constantes que serão usadas ao longo do algoritmo
SOLUTION_BOMB = -1
A_UNKNOWN = RGB(1,1,1)
A_FLAG = RGB(1,0,0)
B_UNKNOWN = 10
B_FLAG = -1

# Variavel auxiliar para exportar as imagens
# image_index = 1
images = []


0-element Array{Any,1}

In [9]:
Base.to_index(A, p::Tuple) = (p[2]-1) * size(A, 1) + p[1]
Base.:+(p::Tuple, q::Tuple)::Tuple = (p[1] + q[1], p[2] + q[2])

in_bounds(p::Tuple, height, width) = 1 <= p[1] <= height && 1 <= p[2] <= width
neighborhood(p::Tuple, it, width, height) = filter(p -> in_bounds(p, width, height), map(dp -> p + dp, it))
moore(p::Tuple, width, height) = neighborhood(p, Iterators.product(-1:1, -1:1), width, height)
moore_expanded(p::Tuple, width, height) = neighborhood(p, Iterators.product(-2:2, -2:2), width, height)
all_cells(height, width) = Iterators.product(1:height, 1:width)

all_cells (generic function with 1 method)

In [4]:
function expand_image(image, scale)
	width, height = size(image)
	new_width = width * scale
	new_height = height * scale

	expanded_image = fill(A_UNKNOWN,new_width, new_height)

	for x in 1:width, y in 1:height
		expanded_image[(x-1)*scale+1:x*scale, (y-1)*scale+1:y*scale] .= image[x,y]
	end
	return expanded_image
end

expand_image (generic function with 1 method)

In [5]:
# Aqui são as funções para implementar o jogo campo minado

function init(bombs, width, height)
	
	solution = zeros(Int, width, height)
	indices = randperm(height*width)[1:bombs]
	solution[indices] .= SOLUTION_BOMB
	for p in all_cells(width, height)
		if solution[p] == SOLUTION_BOMB
			continue
		end
		solution[p] = count(q -> solution[q] == SOLUTION_BOMB, moore(p, width, height))
	end
	return solution
end

function hit(A, solution, p)
	width, height = size(A)
	
	# Se tentar clicar na célula que já foi clicada, a função para
	if !(A[p] in [A_UNKNOWN, A_FLAG])
		return A
	end

	# Se a célula tiver valor 0, clica nas células da vizinhança
	if solution[p] == SOLUTION_BOMB
		# display(solution)
		error("game over")
	else
		value = solution[p]/10
		A[p] = RGB(value, value, value)
		if solution[p] == 0
			for q in moore(p, width, height)
				A = hit(A, solution, q)
			end
		end
	end
	return A
end

function flag(A, solution, p)
	# display("flag $c")
	if A[p] == A_UNKNOWN
		A[p] = A_FLAG
	end
	return A
end

function toggle(A, solution, p)
	if A[p] == A_UNKNOWN
		A[p] = A_FLAG
	elseif A[p] == A_FLAG
		A[p] = A_UNKNOWN
	end
	return A
end

toggle (generic function with 1 method)

In [6]:
function A_to_B(A)
	width, height = size(A)
	B = zeros(width, height)
	for p in all_cells(width, height)
		if A[p] == A_FLAG
			B[p] = B_FLAG
		else
			B[p] = round(A[p].r * 10)
		end
	end
	
	return B
end

function neighbors(p, B)
	width, height = size(B)
    return filter(q -> B[q] in [B_UNKNOWN, B_FLAG], moore(p, width, height))
end

function flagged_neighbors(p, B)
	width, height = size(B)
    return filter(q -> B[q] == B_FLAG, moore(p, width, height))
end

function non_flagged_neighbors(p, B)
	width, height = size(B)
    return filter(q -> B[q] == B_UNKNOWN, moore(p, width, height))
end

function is_solved(B, bombs)
    return count(B .!= B_FLAG) == bombs || all(B .!= B_UNKNOWN)
end

is_solved (generic function with 1 method)

In [15]:

function solve_no_set(A, solution, bombs)
	B = A_to_B(A)
	width, height = size(A)
	while !is_solved(B, bombs)
		old_A = copy(A)
		for p in all_cells(width, height)
			if B[p] in [0, B_UNKNOWN]
				continue
			end
			n = neighbors(p, B)
			if round(B[p]) == length(n)
				for q in n
					if B[q] == B_UNKNOWN
						A = flag(A, solution, q)
						B = A_to_B(A)
						push!(images, copy(A));
					end
				end
			end

			fn = flagged_neighbors(p, B)
			nfn = non_flagged_neighbors(p, B)

			if round(B[p]) == length(fn)
				for q in nfn
					if B[q] == B_UNKNOWN
						A = hit(A, solution, q)
						B = A_to_B(A)
						push!(images, copy(A));
					end
				end
			end
		end
		# Se nenhuma celula foi alterada, entao clicamos numa celula desconhecida as cegas
		if old_A == A
			for p in all_cells(width, height)
				if B[p] == B_UNKNOWN
					A = hit(A, solution, p)
					B = A_to_B(A)
					push!(images, copy(A));
					break
				end
			end
		end
	end
	return A
end


function solve_with_set(A, solution, bombs)
	B = A_to_B(A)
	width, height = size(A)
	while !is_solved(B, bombs)
		old_A = copy(A)
		for p in all_cells(width, height)
			if B[p] in [0, B_UNKNOWN]
				continue
			end
			n = neighbors(p, B)
			if round(B[p]) == length(n)
				for q in n
					if B[q] == B_UNKNOWN
						A = flag(A, solution, q)
						B = A_to_B(A)
						push!(images, copy(A));
					end
				end
			end

			fn = flagged_neighbors(p, B)
			nfn = non_flagged_neighbors(p, B)

			if round(B[p]) == length(fn)
				for q in nfn
					if B[q] == B_UNKNOWN
						A = hit(A, solution, q)
						B = A_to_B(A)
						push!(images, copy(A));
					end
				end
			end
		end
		for p in all_cells(width, height)
				
			if B[p] in [0, B_UNKNOWN]
				continue
			end
			nfn_a = non_flagged_neighbors(p, B)
			fn_a = flagged_neighbors(p, B)
			a_value = B[p] - length(fn_a)
			
			for q in moore_expanded(p, width, height)
				
				if B[q] in [0, B_UNKNOWN]
					continue
				end
				nfn_b = non_flagged_neighbors(q, B)
				fn_b = flagged_neighbors(q, B)
				b_value = B[q] - length(fn_b)
				if a_value - b_value == length(setdiff(nfn_a, nfn_b))
					for r in setdiff(nfn_a, nfn_b)
						if B[r] == B_UNKNOWN
							A = flag(A, solution, r)
							B = A_to_B(A)
							push!(images, copy(A));
						end
					end
					for r in setdiff(nfn_b, nfn_a)
						if B[r] == B_UNKNOWN
							A = hit(A, solution, r)
							B = A_to_B(A)
							push!(images, copy(A));
						end
					end
				end
			end
		end

		# Se nenhuma celula foi alterada, entao clicamos numa celula desconhecida as cegas
		if old_A == A
			for p in all_cells(width, height)
				if B[p] == B_UNKNOWN
					A = hit(A, solution, p)
					B = A_to_B(A)
					push!(images, copy(A));
					break
				end
			end
		end
	end
	return A
end





solve_with_set (generic function with 2 methods)

In [21]:
# solution = [
#   0  0  0   0   0  0  0   1  -1  1;
#   1  1  0   0   0  0  0   1   1  1;
#  -1  1  1   1   1  0  1   1   1  0;
#   1  1  1  -1   2  1  1  -1   1  0;
#   0  0  1   2  -1  1  1   1   1  0
# ];
NUMBER_OF_TESTS = 1000
for instance in [(8,8,10), (16,16,40), (30,16,99)]
  width, height, bombs = instance
  print("Caso $instance com densidade ")
  print(round(bombs/(width*height)*100, digits=2))
  println("%")
  counter_no_set = 0
  counter_with_set = 0

  for i in 1:NUMBER_OF_TESTS
    solution = init(bombs, width, height)
    images = []
    A = fill(A_UNKNOWN, size(solution))
    push!(images, copy(A))
    try
      A = solve_no_set(A, solution, bombs)
    catch e
      counter_no_set += 1
    end
    try
      A = solve_with_set(A, solution, bombs)
    catch e
      counter_with_set += 1
    end
  end
  print("Chance de sucesso básico: ")
  print(round((NUMBER_OF_TESTS-counter_no_set)/NUMBER_OF_TESTS*100, digits=2))
  println("%")
  print("Chance de sucesso avançado: ")
  print(round((NUMBER_OF_TESTS-counter_with_set)/NUMBER_OF_TESTS*100, digits=2))
  println("%\n")

end


Caso (8, 8, 10) com densidade 15.62%


Chance de sucesso básico: 34.4%
Chance de sucesso avançado: 42.1%

Caso (16, 16, 40) com densidade 15.62%


Chance de sucesso básico: 26.2%
Chance de sucesso avançado: 28.7%

Caso (30, 16, 99) com densidade 20.62%


Chance de sucesso básico: 0.9%
Chance de sucesso avançado: 0.9%

